In [5]:
from Vissim_env_class import env
from Agent_class import ACAgent
import numpy as np 

%matplotlib inline

# Balance

In [2]:
model_name  = 'Balance'
vissim_working_directory =  'C:\\Users\\Rzhang\\Desktop\\MLforFlowOptimisationOrigine\\Vissim\\'
sim_length = 3601

# all controller actions
Balance_dictionary =\
{\
    # Controller Number 2 
    0 : {'compatible_actions' : {   0 : [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0],
                                    1 : [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1],
                                    2 : [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0] },
         
         'link' : [2, 40, 7, 38],
         'lane' : ['2-1', '2-2', '2-3', '40-1', '7-1', '7-2', '7-3', '38-1'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [8],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
         
         
        },
    # Controller Number 3
    1 : {'compatible_actions' : {   0 : [0, 1, 0, 0, 1, 0, 1, 1],
                                    1 : [1, 0, 0, 1, 0, 0, 0, 0],
                                    2 : [0, 0, 1, 0, 0, 1, 0, 0] },
         
         'link' : [5, 48, 70, 46],
         'lane' : ['5-1', '5-2', '5-3', '48-1', '70-1', '70-2', '70-3', '46-1'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [8],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
        },
    # Controller Number 4
    2 : {'compatible_actions' : {   0 : [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                    1 : [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                    2 : [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                                    3 : [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
         
         'link' : [73, 100, 84, 95],
         'lane' : ['73-1', '73-2', '73-3', '100-1', '100-2', '100-3', '100-4',\
                  '84-1', '84-2', '84-3', '95-1', '95-2', '95-3', '95-4'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [14],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
        },
    # Controller Number 5
    3 : {'compatible_actions' : {   0 : [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
                                    1 : [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
                                    2 : [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1]},
         
         'link' : [87, 36, 10, 34],
         'lane' : ['87-1', '87-2', '87-3', '36-1', '10-1', '10-2', '10-3', '34-1'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [8],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
        },
    # Controller Number 6 
    4 : {'compatible_actions' : {   0 : [0, 1, 1, 0, 0],
                                    1 : [1, 1, 0, 0, 0],
                                    2 : [0, 0, 0, 1, 0]},
         'link' : [8, 24, 13],
         'lane' : ['8-1', '8-2', '24-1', '13-1', '13-2', '13-3'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [6],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
        },
    # Controller Number 8
    5 : {'compatible_actions' : {   0 : [1, 0, 1, 0, 1, 0],
                                    1 : [0, 1, 0, 1, 0, 1]},
         'link' : [26, 23, 35],
         'lane' : ['26-1', '23-1', '35-1'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [3],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
         
        },
    # Controller Number 9
    6 : {'compatible_actions' : {   0 : [0, 1, 0, 1, 1, 1],
                                    1 : [1, 0, 1, 0, 0, 0]},
         'link' : [51, 92, 64, 19],
         'lane' : ['51-1', '92-1', '92-2', '64-1', '19-1', '19-2'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [6],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
        },
    # Contoller Number 10
    7 : {'compatible_actions' : {   0 : [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
                                    1 : [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                                    2 : [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
         'link' : [18, 66, 16],
         'lane' : ['18-1', '18-2', '18-3', '66-1', '16-1', '16-2', '16-3'],
         
         'controled_by_com' : True,
         'green_time' : 5,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [7],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
        },
    # Controller Number 12
    8 : {'compatible_actions' : {   0 : [1, 0, 1, 0, 0, 0, 0],
                                    1 : [0, 1, 0, 0, 0, 0, 0]},
         'link' : [62, 45, 44],
         'lane' : ['62-1', '45-1', '44-1'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [3],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
        },
    # Controller Number 13
    9 : {'compatible_actions' : {   0 : [0, 1, 0, 1, 1, 0, 1, 0],
                                    1 : [1, 0, 1, 0, 0, 1, 0, 1]},
         'link' : [60, 43, 55, 58],
         'lane' : ['60-1', '43-1', '55-1', '58-1'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [4],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
            
        },
    # Controller 15
    10 : {'compatible_actions' : {  0 : [1, 0, 1, 0, 0, 1, 0, 1],
                                    1 : [0, 1, 0, 1, 1, 0, 1, 0]},
         'link' : [32, 42, 30, 39],
         'lane' : ['32-1', '42-1', '30-1', '39-1'],
         
         'controled_by_com' : True,
         'green_time' : 4,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [4],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
        },
    # Controller 16
    11 : {'compatible_actions' :  { 0 : [1, 0, 1, 0, 0, 1, 0, 1],
                                    1 : [0, 1, 0, 1, 1, 0, 1, 0]},
         'link' : [29, 50, 28, 47],
         'lane' : ['29-1', '50-1', '28-1', '47-1'],
         
         'controled_by_com' : True,
         'green_time' : 8,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [4],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
        },
    # Controller 17
    12 : {'compatible_actions' :  { 0 : [1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1],
                                    1 : [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1],
                                    2 : [0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0],
                                    3 : [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0]},
         'link' : [27, 22, 25, 77],
         'lane' : ['27-1', '22-1', '22-2', '22-3', '25-1', '77-1', '77-2'],
         
         'controled_by_com' : True,
         'green_time' : 7,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [7],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
            
        },
    # Controller 33 
    13 : {'compatible_actions' :  { 0 : [1, 0, 0, 1, 0, 0, 1, 0, 0],
                                    1 : [0, 0, 1, 1, 0, 1, 0, 0, 0],
                                    2 : [0, 1, 0, 0, 1, 1, 0, 1, 1]},
         'link' : [68, 71, 75],
         'lane' : ['68-1', '68-2', '68-3', '71-1', '71-2', '75-1'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 1,
         'state_size' : [6],
         'state_type' : 'Queues',
         'reward_type' : 'Queues'
        }
}

In [ ]:
env = env(model_name, vissim_working_directory, sim_length, Balance_dictionnary,\
            timesteps_per_second = 1, mode = 'training', delete_results = True, verbose = True)

Working Directory set to: C:\Users\Rzhang\Desktop\MLforFlowOptimisationOrigine\Vissim\
Generating Cache...


In [4]:
env.SCUs

{0: <Vissim_SCU_class.Signal_Control_Unit at 0x17767ec6be0>,
 1: <Vissim_SCU_class.Signal_Control_Unit at 0x17767ec6e48>,
 2: <Vissim_SCU_class.Signal_Control_Unit at 0x17767ed6978>,
 3: <Vissim_SCU_class.Signal_Control_Unit at 0x17767ee4470>,
 4: <Vissim_SCU_class.Signal_Control_Unit at 0x17767eec7b8>,
 5: <Vissim_SCU_class.Signal_Control_Unit at 0x17767eec6d8>,
 6: <Vissim_SCU_class.Signal_Control_Unit at 0x17767efab38>,
 7: <Vissim_SCU_class.Signal_Control_Unit at 0x17767efa278>,
 8: <Vissim_SCU_class.Signal_Control_Unit at 0x17767f07a20>,
 9: <Vissim_SCU_class.Signal_Control_Unit at 0x17767f070b8>,
 10: <Vissim_SCU_class.Signal_Control_Unit at 0x17767f14a20>,
 11: <Vissim_SCU_class.Signal_Control_Unit at 0x17767f14518>,
 12: <Vissim_SCU_class.Signal_Control_Unit at 0x17767f1b860>,
 13: <Vissim_SCU_class.Signal_Control_Unit at 0x17767f1bef0>}

In [5]:
env.SCUs[0].state

array([[0., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
#env.reset()

In [7]:
actions = dict()
for i in range(len(env.SCUs)):
    actions[i]=0
    
print(actions)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0}


In [8]:
class Cyclic_Control():
    def __init__(self,size):
        self.action = 0
        self.size = size
        
    def choose_action(self,state=None):
        self.action = (self.action + 1) % self.size
        return self.action
CC = [] 

for idx, info in Balance_dictionnary.items():
        cycle_size = len(info['compatible_actions'])
        CC.append(Cyclic_Control(cycle_size))

In [9]:
start_state = env.get_state()
for idx, s in start_state.items():
    actions[idx] = CC[idx].choose_action(s)
    
for _ in range(10000):
    action_required, SARSDs = env.step(actions)
    if action_required : 
        actions = dict()
        for idx , sarsd in SARSDs.items():
            s,a,r,ns,d = sarsd
            #print(sarsd)
            
            # in order to find the next action you need to evaluate the "next_state" because it is the current state of the simulator
            actions[idx] = CC[idx].choose_action(ns)
        
    if env.done :
        env.reset()
        for idx, s in start_state.items():
            actions[idx] = CC[idx].choose_action(ns)
        
        

com_error: (-2147352567, 'Une exception s’est produite.', (0, 'VISSIM.Vissim.1100', 'AttValue failed: Object 1 - 3: SG2: Attribute Signal state is no subject to changes.', None, 0, -2147352567), None)

In [ ]:
# not bad with the first model. need a fonction to decrease entropy now 

gamma = 0.85
alpha = 0.0005
value = 25
entropy = 5000
n_step_size = 11
state_size = [12]
reduce_entropy_every = 1000
entropy_threshold = 0.5


# for the monitoring
horizon = 50
n_sample = 10

Agents = []

for idx, info in Balance_dictionary.items():
        acts = info['compatible_actions']
        Agent = ACAgent(state_size, len(acts), idx, n_step_size, gamma, alpha, entropy, value)
        Agents.append(Agent)

In [ ]:
start_state = env.get_state()
actions = {}
for idx, s in start_state.items():
            actions[idx] = Agents[idx].choose_action(s)

for i in range(30000):
    action_required, SARSDs = env.step_to_next_action(actions)
    if action_required : 
        actions = dict()
        for idx , sarsd in SARSDs.items():
            s,a,r,ns,d = sarsd
            
            #print(sarsd)
            Agents[idx].remember(s,a,r,ns,d)
            if len(Agents[idx].memory) >= Agents[idx].n_step_size :
                Agents[idx].learn() 
            
            # in order to find the next action you need to evaluate the "next_state" because it is the current state of the simulator
            actions[idx] = int(Agents[idx].choose_action(ns))
            #print(actions)
            
            if (i+1)%reduce_entropy_every == 0:
                if Agents[idx].params['entropy'] >= entropy_threshold :
                    Agents[idx].reduce_entropy()
                    print ("Agent {} : Entropy reduced to {} " .format(idx, Agents[idx].params['entropy']))
        
    # For the saving , monitoring of the agent
    if env.done :
        env.reset()
        for idx, agent in enumerate(Agents):
                    predicted_values, true_values, proba0, probas = agent.value_check(horizon, n_sample)
                    print ("Agent {} : Predicted Values and True Return : \n {} \n {}" .format(idx, predicted_values, true_values))
                    print ("Agent {} : Proba distribution on those states : \n {}" .format(idx, probas))
                    print ("Agent {} : Proba distribution on the 0 state : \n {}" .format(idx, proba0))
                    agent.reset()
                    
        
        for idx, s in start_state.items():
            actions[idx] = Agents[idx].choose_action(s)

In [20]:
for i in range(14):
    print(len(env.SCUs[i].compatible_actions))



3
3
4
3
3
2
2
3
2
2
2
2
4
3


# Single_Cross_Straight

---> The lack of speed comes from the size of the model (particularly the change of color of the heads). 

In [2]:
model_name  = 'Single_Cross_Straight'
vissim_working_directory =  'C:\\Users\\Rzhang\\Desktop\\MLforFlowOptimisationOrigine\\Vissim\\'
sim_length = 3601

# all controller actions
Single_Cross_Straight_dictionary =\
{\
    # Controller Number 0 
    0 : {'compatible_actions' : {     0 : [1, 0, 1, 0],
                                     1 : [0, 1, 0, 1]
        },
 
         'link' : [1, 3, 5, 7],
         'lane' : ['1-1', '3-1', '5-1', '7-1'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 0,
         'state_size' : [4],
         'state_type' : 'Queues',
         'reward_type' : 'Queues' 
}
}

In [3]:
env = env(model_name, vissim_working_directory, sim_length, Single_Cross_Straight_dictionary,\
            timesteps_per_second = 1, mode = 'training', delete_results = True, verbose = True)

env.SCUs


Working Directory set to: C:\Users\Rzhang\Desktop\MLforFlowOptimisationOrigine\Vissim\
Generating Cache...
Cache generated.

****************************
*   COM Server dispatched  *
****************************

Attempting to load Model File: Single_Cross_Straight.inpx ...
Load process successful
Simulation length set to 3601 seconds.
Results from Previous Simulations: Deleted. Fresh Start Available.
Fetched and containerized Simulation Object
Fetched and containerized Network Object 

*******************************************************
*                                                     *
*                 SETUP COMPLETE                      *
*                                                     *
*******************************************************

0.07547640800476074


{0: <Vissim_SCU_class.Signal_Control_Unit at 0x1e6913145c0>}

In [4]:
gamma = 0.85
alpha = 0.00005
ID = 0
value = 25
entropy = 0.05
n_step_size = 16
state_size = [4]


# for the monitoring
horizon = 100
n_sample = 10

Agents = []

for idx, info in Single_Cross_Straight_dictionary.items():
        acts = info['compatible_actions']
        Agent = ACAgent(state_size, len(acts), ID, n_step_size, gamma, alpha, entropy, value)
        Agents.append(Agent)


Deploying instance of Actor_Critic Agent(s) !!! TENSORFLOW 2 IS NEEDED !!! 
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  210       
_________________________________________________________________
dense_1 (Dense)              multiple                  210       
_________________________________________________________________
value (Dense)                multiple                  43        
_________________________________________________________________
policy_logits (Dense)        multiple                  86        
_________________________________________________________________
probability_distribution (Pr multiple                  0         
Total params: 549
Trainable params: 549
Non-trainable params: 0
_________________________________________________________________
To be corrected


In [ ]:
start_state = env.get_state()
actions = {}
for idx, s in start_state.items():
            actions[idx] = Agents[idx].choose_action(s)

for _ in range(100000):
    action_required, SARSDs = env.step_to_next_action(actions)
    if action_required : 
        actions = dict()
        for idx , sarsd in SARSDs.items():
            s,a,r,ns,d = sarsd
            
            #print(sarsd)
            Agents[idx].remember(s,a,r,ns,d)
            if len(Agents[idx].memory) >= Agents[idx].n_step_size :
                Agents[idx].learn() 
            
            # in order to find the next action you need to evaluate the "next_state" because it is the current state of the simulator
            actions[idx] = int(Agents[idx].choose_action(ns))
            #print(actions)
            
    
    # For the saving , monitoring of the agent
    if env.done :
        env.reset()
        for idx, agent in enumerate(Agents):
                    predicted_values, true_values, proba0, probas = agent.value_check(horizon, n_sample)
                    print ("Agent {} : Predicted Values and True Return : \n {} \n {}" .format(idx, predicted_values, true_values))
                    print ("Agent {} : Proba distribution on those states : \n {}" .format(idx, probas))
                    print ("Agent {} : Proba distribution on the 0 state : \n {}" .format(idx, proba0))
                    agent.reset()
                    
        
        for idx, s in start_state.items():
            actions[idx] = Agents[idx].choose_action(s)
            
            
            
        
            

Agent 0 : Predicted Values and True Return : 
 [-13.0, -13.0, -13.0, -13.0, -13.0, -13.0, -13.0, -13.0, -13.0, -13.0] 
 [-527.0, -527.0, -527.0, -527.0, -527.0, -527.0, -527.0, -527.0, -527.0, -527.0]
Agent 0 : Proba distribution on those states : 
 [[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]]
Agent 0 : Proba distribution on the 0 state : 
 [0.5 0.5]
Agent 0 : Predicted Values and True Return : 
 [-31.0, -31.0, -31.0, -31.0, -31.0, -31.0, -12.0, -31.0, -31.0, -31.0] 
 [-527.0, -527.0, -527.0, -527.0, -527.0, -527.0, -86.0, -527.0, -301.0, -527.0]
Agent 0 : Proba distribution on those states : 
 [[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]]
Agent 0 : Proba distribution on the 0 state : 
 [0.53 0.47]
Agent 0 : Predicted Values and True Return : 
 [-34.0, -49.0, -23.0, -49.0, -43.0, -15.0, -49.0, -49.0, -49.0, -24.0] 
 [-54.0, -527.0, -527.

Agent 0 : Predicted Values and True Return : 
 [-25.0, -13.0, -20.0, -31.0, -42.0, -25.0, -16.0, -5.0, -14.0, -5.0] 
 [-527.0, -527.0, -527.0, -527.0, -37.0, -527.0, -527.0, -527.0, -527.0, -527.0]
Agent 0 : Proba distribution on those states : 
 [[0.0, 1.0], [0.0, 1.0], [0.94, 0.06], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.37, 0.63], [0.0, 1.0], [0.37, 0.63]]
Agent 0 : Proba distribution on the 0 state : 
 [0.37 0.63]
Agent 0 : Predicted Values and True Return : 
 [-29.0, -20.0, -24.0, -6.0, -10.0, -18.0, -20.0, -6.0, -9.0, -6.0] 
 [-527.0, -228.0, -527.0, -527.0, -527.0, -527.0, -35.0, -527.0, -527.0, -527.0]
Agent 0 : Proba distribution on those states : 
 [[1.0, 0.0], [0.94, 0.06], [0.0, 1.0], [0.43, 0.57], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.43, 0.57], [0.0, 1.0], [0.43, 0.57]]
Agent 0 : Proba distribution on the 0 state : 
 [0.43 0.57]
Agent 0 : Predicted Values and True Return : 
 [-20.0, -10.0, -24.0, -14.0, -17.0, -14.0, -19.0, -13.0, -6.0, -13.0] 
 [-323.0, -453

In [7]:
env.done


False

# Single_Cross_Triple 4 actions

In [6]:
model_name  = 'Single_Cross_Triple'
vissim_working_directory =  'C:\\Users\\Rzhang\\Desktop\\MLforFlowOptimisationOrigine\\Vissim\\'
sim_length = 2000

# all controller actions
Single_Cross_Triple_dictionary4 =\
{\
    # Controller Number 0 
    0 : {'compatible_actions' : {    0 : [1,1,1,0,0,0,0,0,0,0,0,0],
                                     1 : [0,0,0,1,1,1,0,0,0,0,0,0],
                                     2 : [0,0,0,0,0,0,1,1,1,0,0,0],
                                     3 : [0,0,0,0,0,0,0,0,0,1,1,1]},
 
         'link' : [1, 3, 5, 7],
         'lane' : ['1-1', '1-2', '1-3', '3-1', '3-2', '3-3', '5-1', '5-2', '5-3', '7-1', '7-2', '7-3'],
         
         'controled_by_com' : True,
         'green_time' : 10,
         'redamber_time' : 1,
         'amber_time' : 3, 
         'red_time' : 0,
         'state_size' : [12],
         'state_type' : 'Queues',
         'reward_type' : 'Queues' 
         }
}

In [7]:
env = env(model_name, vissim_working_directory, sim_length, Single_Cross_Triple_dictionary4,\
            timesteps_per_second = 1, mode = 'training', delete_results = True, verbose = True)

env.SCUs

Working Directory set to: C:\Users\Rzhang\Desktop\MLforFlowOptimisationOrigine\Vissim\
Generating Cache...
Cache generated.

****************************
*   COM Server dispatched  *
****************************

Attempting to load Model File: Single_Cross_Triple.inpx ...
Load process successful
Simulation length set to 2000 seconds.
Results from Previous Simulations: Deleted. Fresh Start Available.
Fetched and containerized Simulation Object
Fetched and containerized Network Object 

*******************************************************
*                                                     *
*                 SETUP COMPLETE                      *
*                                                     *
*******************************************************

0.12905240058898926


{0: <Vissim_SCU_class.Signal_Control_Unit at 0x1ba7bfc0e48>}

In [9]:
# not bad with the first model. need a fonction to decrease entropy now 

gamma = 0.85
alpha = 0.0005
ID = 0
value = 25
entropy = 5000
n_step_size = 11
state_size = [12]
reduce_entropy_every = 1000
entropy_threshold = 0.5


# for the monitoring
horizon = 50
n_sample = 10

Agents = []

for idx, info in Single_Cross_Triple_dictionary4.items():
        acts = info['compatible_actions']
        Agent = ACAgent(state_size, len(acts), ID, n_step_size, gamma, alpha, entropy, value)
        Agents.append(Agent)

Deploying instance of Actor_Critic Agent(s) !!! TENSORFLOW 2 IS NEEDED !!! 
Model: "model2_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
value1 (Dense)               multiple                  832       
_________________________________________________________________
value2 (Dense)               multiple                  4160      
_________________________________________________________________
value3 (Dense)               multiple                  2730      
_________________________________________________________________
value (Dense)                multiple                  43        
_________________________________________________________________
policy_logits1 (Dense)       multiple                  546       
_________________________________________________________________
policy_logits2 (Dense)       multiple                  1806      
________________________________________________

In [10]:
start_state = env.get_state()
actions = {}
for idx, s in start_state.items():
            actions[idx] = Agents[idx].choose_action(s)

for i in range(30000):
    action_required, SARSDs = env.step_to_next_action(actions)
    if action_required : 
        actions = dict()
        for idx , sarsd in SARSDs.items():
            s,a,r,ns,d = sarsd
            
            #print(sarsd)
            Agents[idx].remember(s,a,r,ns,d)
            if len(Agents[idx].memory) >= Agents[idx].n_step_size :
                Agents[idx].learn() 
            
            # in order to find the next action you need to evaluate the "next_state" because it is the current state of the simulator
            actions[idx] = int(Agents[idx].choose_action(ns))
            #print(actions)
            
            if (i+1)%reduce_entropy_every == 0:
                if Agents[idx].params['entropy'] >= entropy_threshold :
                    Agents[idx].reduce_entropy()
                    print ("Agent {} : Entropy reduced to {} " .format(idx, Agents[idx].params['entropy']))
        
    # For the saving , monitoring of the agent
    if env.done :
        env.reset()
        for idx, agent in enumerate(Agents):
                    predicted_values, true_values, proba0, probas = agent.value_check(horizon, n_sample)
                    print ("Agent {} : Predicted Values and True Return : \n {} \n {}" .format(idx, predicted_values, true_values))
                    print ("Agent {} : Proba distribution on those states : \n {}" .format(idx, probas))
                    print ("Agent {} : Proba distribution on the 0 state : \n {}" .format(idx, proba0))
                    agent.reset()
                    
        
        for idx, s in start_state.items():
            actions[idx] = Agents[idx].choose_action(s)

Agent 0 : Predicted Values and True Return : 
 [-111.0, -180.0, -156.0, -179.0, -78.0, -47.0, -78.0, -106.0, -111.0, -143.0] 
 [-216.0, -274.0, -253.0, -351.0, -214.0, -223.0, -214.0, -240.0, -256.0, -220.0]
Agent 0 : Proba distribution on those states : 
 [[0.28, 0.27, 0.22, 0.23], [0.32, 0.26, 0.23, 0.18], [0.28, 0.23, 0.27, 0.22], [0.28, 0.27, 0.24, 0.2], [0.34, 0.26, 0.2, 0.19], [0.26, 0.24, 0.25, 0.25], [0.34, 0.26, 0.2, 0.19], [0.23, 0.27, 0.25, 0.25], [0.29, 0.27, 0.24, 0.2], [0.3, 0.26, 0.23, 0.21]]
Agent 0 : Proba distribution on the 0 state : 
 [0.25 0.25 0.25 0.25]
Agent 0 : Predicted Values and True Return : 
 [-594.0, -339.0, -359.0, -315.0, -472.0, -67.0, -211.0, -11.0, -432.0, -450.0] 
 [-240.0, -253.0, -274.0, -278.0, -198.0, -223.0, -380.0, -148.0, -273.0, -239.0]
Agent 0 : Proba distribution on those states : 
 [[0.23, 0.25, 0.26, 0.26], [0.25, 0.23, 0.25, 0.27], [0.24, 0.24, 0.26, 0.26], [0.25, 0.24, 0.25, 0.27], [0.24, 0.22, 0.27, 0.27], [0.24, 0.25, 0.25, 0.25], [0

Agent 0 : Predicted Values and True Return : 
 [-617.0, -748.0, -616.0, -689.0, -581.0, -599.0, -169.0, -586.0, -507.0, -218.0] 
 [-212.0, -210.0, -230.0, -238.0, -248.0, -218.0, -373.0, -230.0, -288.0, -318.0]
Agent 0 : Proba distribution on those states : 
 [[0.26, 0.24, 0.23, 0.26], [0.27, 0.23, 0.28, 0.22], [0.23, 0.25, 0.26, 0.25], [0.27, 0.24, 0.24, 0.24], [0.29, 0.21, 0.25, 0.25], [0.28, 0.21, 0.24, 0.27], [0.27, 0.25, 0.25, 0.24], [0.28, 0.26, 0.23, 0.23], [0.26, 0.23, 0.24, 0.28], [0.26, 0.26, 0.23, 0.25]]
Agent 0 : Proba distribution on the 0 state : 
 [0.27 0.24 0.25 0.24]
Model: "model2_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
value1 (Dense)               multiple                  832       
_________________________________________________________________
value2 (Dense)               multiple                  4160      
_____________________________________________________________

Agent 0 : Predicted Values and True Return : 
 [-326.0, -83.0, -462.0, -265.0, -182.0, -145.0, -147.0, -161.0, -119.0, -113.0] 
 [-240.0, -240.0, -261.0, -271.0, -316.0, -210.0, -377.0, -212.0, -248.0, -261.0]
Agent 0 : Proba distribution on those states : 
 [[0.01, 0.0, 0.0, 0.98], [0.0, 0.05, 0.81, 0.14], [0.0, 0.0, 0.0, 1.0], [0.0, 0.07, 0.0, 0.93], [0.1, 0.76, 0.04, 0.1], [0.33, 0.03, 0.34, 0.3], [0.26, 0.06, 0.08, 0.6], [0.13, 0.13, 0.0, 0.74], [0.0, 0.04, 0.94, 0.02], [0.11, 0.07, 0.65, 0.17]]
Agent 0 : Proba distribution on the 0 state : 
 [0.4  0.14 0.31 0.15]
Agent 0 : Predicted Values and True Return : 
 [-1199.0, -631.0, -1137.0, -1368.0, -1242.0, -1724.0, -1555.0, -1664.0, -1844.0, -1475.0] 
 [-218.0, -261.0, -238.0, -218.0, -240.0, -203.0, -241.0, -183.0, -181.0, -236.0]
Agent 0 : Proba distribution on those states : 
 [[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.36, 0.64], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.98, 0.02], [0.0, 0.0, 0.99, 0.01], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0

KeyboardInterrupt: 

In [6]:
Agents[0].model.loss

[<bound method ACAgent._logits_loss of <Agent_class.ACAgent object at 0x000002BA33964AC8>>,
 <bound method ACAgent._value_loss of <Agent_class.ACAgent object at 0x000002BA33964AC8>>]

# Single_Cross_Triple 8 actions

In [7]:
model_name  = 'Single_Cross_Triple'
vissim_working_directory =  'C:\\Users\\Rzhang\\Desktop\\MLforFlowOptimisationOrigine\\Vissim\\'
sim_length = 3601

# all controller actions
Single_Cross_Triple_dictionary8 =\
{\
    # Controller Number 0 
    0 : {'compatible_actions' : {    0 : [1,1,1,0,0,0,0,0,0,0,0,0],
                                     1 : [0,0,0,1,1,1,0,0,0,0,0,0],
                                     2 : [0,0,0,0,0,0,1,1,1,0,0,0],
                                     3 : [0,0,0,0,0,0,0,0,0,1,1,1],
                                     4 : [1,0,0,0,0,0,1,0,0,0,0,0],
                                     5 : [0,0,0,1,0,0,0,0,0,1,0,0],
                                     6 : [0,1,1,0,0,0,0,1,1,0,0,0],
                                     7 : [0,0,0,0,1,1,0,0,0,0,1,1]},
 
         'link' : [1, 3, 5, 7],
         'lane' : ['1-1', '1-2', '1-3', '3-1', '3-2', '3-3', '5-1', '5-2', '5-3', '7-1', '7-2', '7-3'],
         
         'controled_by_com' : True,
         'green_time' : 6,
         'redamber_time' : 0,
         'amber_time' : 3, 
         'red_time' : 0,
         'state_size' : [12],
         'state_type' : 'Queues',
         'reward_type' : 'Queues' 
         }
}

In [8]:
env = env(model_name, vissim_working_directory, sim_length, Single_Cross_Triple_dictionary8,\
            timesteps_per_second = 1, mode = 'training', delete_results = True, verbose = True)

env.SCUs

Working Directory set to: C:\Users\Rzhang\Desktop\MLforFlowOptimisationOrigine\Vissim\
Generating Cache...
Cache generated.

****************************
*   COM Server dispatched  *
****************************

Attempting to load Model File: Single_Cross_Triple.inpx ...
Failed load attempt 1/5. Re-attempting.
Working Directory set to: C:\Users\Rzhang\Desktop\MLforFlowOptimisationOrigine\Vissim\
Generating Cache...
Cache generated.

****************************
*   COM Server dispatched  *
****************************

Attempting to load Model File: Single_Cross_Triple.inpx ...
Load process successful
Simulation length set to 3601 seconds.
Results from Previous Simulations: Deleted. Fresh Start Available.
Fetched and containerized Simulation Object
Fetched and containerized Network Object 

*******************************************************
*                                                     *
*                 SETUP COMPLETE                      *
*                            

{0: <Vissim_SCU_class.Signal_Control_Unit at 0x1bbe8d2ca58>}

In [9]:
gamma = 0.85
alpha = 0.00005
ID = 0
value = 25
entropy = 2000
n_step_size = 16
state_size = [12]
reduce_entropy_every = 100



# for the monitoring
horizon = 100
n_sample = 10

Agents = []

for idx, info in Single_Cross_Triple_dictionary8.items():
        acts = info['compatible_actions']
        Agent = ACAgent(state_size, len(acts), ID, n_step_size, gamma, alpha, entropy, value)
        Agents.append(Agent)

Deploying instance of Actor_Critic Agent(s) !!! TENSORFLOW 2 IS NEEDED !!! 
Model: "model2_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
value1 (Dense)               multiple                  780       
_________________________________________________________________
value2 (Dense)               multiple                  1281      
_________________________________________________________________
value3 (Dense)               multiple                  462       
_________________________________________________________________
value (Dense)                multiple                  22        
_________________________________________________________________
policy_logits1 (Dense)       multiple                  780       
_________________________________________________________________
policy_logits2 (Dense)       multiple                  1281      
________________________________________________

In [10]:
start_state = env.get_state()
actions = {}
for idx, s in start_state.items():
            actions[idx] = Agents[idx].choose_action(s)

for i in range(100000):
    action_required, SARSDs = env.step_to_next_action(actions)
    if action_required : 
        actions = dict()
        for idx , sarsd in SARSDs.items():
            s,a,r,ns,d = sarsd
            
            #print(sarsd)
            Agents[idx].remember(s,a,r,ns,d)
            if len(Agents[idx].memory) >= Agents[idx].n_step_size :
                Agents[idx].learn() 
            
            # in order to find the next action you need to evaluate the "next_state" because it is the current state of the simulator
            actions[idx] = int(Agents[idx].choose_action(ns))
            #print(actions)
            
            if (i+1)%reduce_entropy_every == 0:
                Agents[idx].reduce_entropy()
                print ("Agent {} : Entropy reduced to {} " .format(idx, Agents[idx].params['entropy']))
            
    
    # For the saving , monitoring of the agent
    if env.done :
        env.reset()
        for idx, agent in enumerate(Agents):
                    predicted_values, true_values, proba0, probas = agent.value_check(horizon, n_sample)
                    print ("Agent {} : Predicted Values and True Return : \n {} \n {}" .format(idx, predicted_values, true_values))
                    print ("Agent {} : Proba distribution on those states : \n {}" .format(idx, probas))
                    print ("Agent {} : Proba distribution on the 0 state : \n {}" .format(idx, proba0))
                    agent.reset()
                    
        
        for idx, s in start_state.items():
            actions[idx] = Agents[idx].choose_action(s)

Agent 0 : Entropy reduced to 1900.0 
Agent 0 : Entropy reduced to 1805.0 
Agent 0 : Entropy reduced to 1714.75 
Agent 0 : Predicted Values and True Return : 
 [-9.0, -16.0, -17.0, -19.0, -19.0, -11.0, -9.0, -10.0, -19.0, -12.0] 
 [-1282.0, -1466.0, -1467.0, -1512.0, -1508.0, -1245.0, -1169.0, -1294.0, -1508.0, -1324.0]
Agent 0 : Proba distribution on those states : 
 [[0.16, 0.0, 0.31, 0.0, 0.42, 0.06, 0.04, 0.0], [0.25, 0.0, 0.1, 0.03, 0.43, 0.13, 0.06, 0.0], [0.23, 0.0, 0.14, 0.05, 0.28, 0.2, 0.09, 0.0], [0.14, 0.0, 0.28, 0.1, 0.1, 0.26, 0.11, 0.0], [0.14, 0.0, 0.35, 0.11, 0.08, 0.22, 0.1, 0.0], [0.26, 0.0, 0.15, 0.01, 0.45, 0.08, 0.05, 0.0], [0.19, 0.0, 0.39, 0.01, 0.16, 0.15, 0.1, 0.0], [0.14, 0.0, 0.21, 0.0, 0.57, 0.05, 0.04, 0.0], [0.14, 0.0, 0.35, 0.11, 0.08, 0.22, 0.1, 0.0], [0.16, 0.0, 0.07, 0.0, 0.68, 0.06, 0.03, 0.0]]
Agent 0 : Proba distribution on the 0 state : 
 [0.13 0.13 0.12 0.12 0.12 0.13 0.13 0.13]
Agent 0 : Entropy reduced to 1629.0124999999998 
Agent 0 : Entropy re

KeyboardInterrupt: 